**# Introduction**

* The next project is based on the kaggle dataset "Logistic regression To predict heart disease" and you can find it in this [link](https://www.kaggle.com/dileep070/heart-disease-prediction-using-logistic-regression)

* The goal of this project is to have a better understanding on the factors that rise the probability of heart disease.

* The features are correctly described in the kaggle dataset.

* We will use some Logistic models in order to predict the probability of have heart problems.

#Let's import our libraries


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns 


Let's collect the data from kaggle


In [6]:
!pip install -q kaggle

In [7]:
#Choice our kaggle.json downloaded file
from google.colab import files
files.upload()

#Make directory named kaggle and copy kaggle.json file there.
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/


Saving kaggle.json to kaggle (4).json


{'kaggle.json': b'{"username":"jesusvaldibiezomora","key":"8cc4e22eb745a9377ad587c057401845"}'}

In [10]:
 ! chmod 600 ~/.kaggle/kaggle.json #Change the permissions of the file.

In [11]:
! kaggle datasets list #- That's all ! You can check if everything's okay by running this command.

ref                                                               title                                                 size  lastUpdated          downloadCount  
----------------------------------------------------------------  --------------------------------------------------  ------  -------------------  -------------  
christianlillelund/donald-trumps-rallies                          Donald Trump's Rallies                               720KB  2020-09-26 10:25:08            738  
heeraldedhia/groceries-dataset                                    Groceries dataset                                    257KB  2020-09-17 04:36:08           3497  
andrewmvd/trip-advisor-hotel-reviews                              Trip Advisor Hotel Reviews                             5MB  2020-09-30 08:31:20           2002  
balraj98/stanford-background-dataset                              Stanford Background Dataset                           17MB  2020-09-26 12:57:59            210  
nehaprabhavalkar/india

In [30]:

#! kaggle datasets download -d dileep070/heart-disease-prediction-using-logistic-regression

! mkdir Logistic_reg_heart_disease_project_dataset
#! cp heart-disease-prediction-using-logistic-regression.zip /Logistic_reg_heart_disease_project_datase/
! unzip heart-disease-prediction-using-logistic-regression.zip -d Logistic_reg_heart_disease_project_dataset


mkdir: cannot create directory ‘Logistic_reg_heart_disease_project_dataset’: File exists
Archive:  heart-disease-prediction-using-logistic-regression.zip
replace Logistic_reg_heart_disease_project_dataset/framingham.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [33]:
!echo $PATH

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin


In [39]:
path = '/Logistic_reg_heart_disease_project_dataset/heart-disease-prediction-using-logistic-regression.csv'

In [40]:
pd.read_csv(path)

FileNotFoundError: ignored